In [2]:
import numpy as np
import scipy as sp
import math
import itertools

In [3]:
cn_train = np.loadtxt('DS2_train_0.txt', delimiter=',', usecols = range(20))
cp_train = np.loadtxt('DS2_train_1.txt', delimiter=',', usecols = range(20))
cn_test = np.loadtxt('DS2_test_0.txt', delimiter=',', usecols = range(20))
cp_test = np.loadtxt('DS2_test_1.txt', delimiter=',', usecols = range(20))

In [4]:
#parameter of gda
pi = (cn_train.shape[0])/(cn_train.shape[0]+cp_train.shape[0])
mu_n = np.mean(cn_train, axis=0)
mu_p = np.mean(cp_train, axis=0)
sum = np.zeros((20,20))
for i in cn_train:
    sum = sum + np.atleast_2d(i-mu_n).T.dot(np.atleast_2d(i-mu_n))
s_1 = sum/cn_train.shape[0]
sum = np.zeros((20,20))
for i in cp_train:
    sum = sum + np.atleast_2d(i-mu_p).T.dot(np.atleast_2d(i-mu_p))
s_2 = sum/cp_train.shape[0]
sigma = pi*s_1 + (1-pi)*s_2

In [5]:
#coefficient of gda
#using coefficient method
w = np.linalg.inv(sigma).dot(np.atleast_2d(mu_n-mu_p).T)
w_0 = -0.5 * np.atleast_2d(mu_n).dot(np.linalg.inv(sigma)).dot(np.atleast_2d(mu_n).T) + 0.5 * np.atleast_2d(mu_p).dot(np.linalg.inv(sigma)).dot(np.atleast_2d(mu_p).T) + math.log((pi/(1-pi)),math.e)

In [6]:
def p(x):
    return (w.T.dot(np.atleast_2d(x).T)+w_0)[0][0]

In [7]:
#test of gda
num_correct = 0
true_positive = 0
false_positive = 0
false_negative = 0
for i in cn_test:
    if p(i) > 0.5:
        num_correct = num_correct + 1
    else:
        false_positive = false_positive + 1
for i in cp_test:
    if p(i) <= 0.5:
        num_correct = num_correct + 1
        true_positive = true_positive + 1
    else:
        false_negative = false_negative + 1
accuracy = num_correct/(cn_test.shape[0]+cp_test.shape[0])
precision = true_positive/(true_positive+false_positive)
recall = true_positive/(true_positive+false_negative)
f_measure = (2*precision*recall)/(precision + recall)
print(accuracy)
print(precision)
print(recall)
print(f_measure)
print(w)
print(w_0)

0.5083333333333333
0.5042087542087542
0.9983333333333333
0.6700223713646533
[[-0.10832542]
 [ 0.01236988]
 [ 0.01566538]
 [-0.01465766]
 [ 0.02018749]
 [ 0.04518463]
 [-0.07808383]
 [ 0.08387873]
 [ 0.02135773]
 [ 0.06952269]
 [ 0.09384125]
 [ 0.03465218]
 [ 0.02263593]
 [-0.00173435]
 [-0.09890626]
 [ 0.05135079]
 [-0.0082887 ]
 [-0.03538724]
 [-0.06158313]
 [-0.01939665]]
[[-0.0499968]]


In [8]:
#define functions of k-NN
def distance(x0, x1):
    return np.linalg.norm(x0-x1)

def get_dist_table(x):
    rank = []
    for i in cn_train:
        rank.append((i, 1, distance(x,i)))
    for i in cp_train:
        rank.append((i, 0, distance(x,i)))
    rank.sort(key = lambda t:t[2])
    return rank

def y_predict(index, k):
    rank = test_dict[index]
    sum = 0
    for i in range(k):
        sum = sum + rank[i][1]
    return sum/k

In [9]:
#load distance table first ty apply dynamic programming
test_dict = []
for i in cn_test:
    test_dict.append(get_dist_table(i))
for i in cp_test:
    test_dict.append(get_dist_table(i))

In [10]:
def test(k):
    num_correct = 0
    true_positive = 0
    false_positive = 0
    false_negative = 0
    index = 0
    for i in cn_test:
        if y_predict(index, k) > 0.5:
            num_correct = num_correct + 1
        else:
            false_positive = false_positive + 1
        index = index + 1
    for i in cp_test:
        if y_predict(index, k) <= 0.5:
            num_correct = num_correct + 1
            true_positive = true_positive + 1
        else:
            false_negative = false_negative + 1
        index = index + 1
    accuracy = num_correct/(cn_test.shape[0]+cp_test.shape[0])
    precision = true_positive/(true_positive+false_positive)
    recall = true_positive/(true_positive+false_negative)
    f_measure = (2*precision*recall)/(precision + recall)
    return (k, accuracy, precision, recall, f_measure)

In [11]:
#test of k-NN
test_table = []
for k in itertools.chain(range(1,11), range(20,101,10), range(200,2701,100)):
    test_table.append(test(k))
test_table.sort(key = lambda t:t[4],reverse=True)
test_table[:10]

[(2100, 0.5, 0.5, 1.0, 0.6666666666666666),
 (2200, 0.5, 0.5, 1.0, 0.6666666666666666),
 (2300, 0.5, 0.5, 1.0, 0.6666666666666666),
 (2400, 0.5, 0.5, 1.0, 0.6666666666666666),
 (2000,
  0.49916666666666665,
  0.49958298582151794,
  0.9983333333333333,
  0.6659255141745414),
 (1900, 0.4975, 0.49873417721518987, 0.985, 0.6621848739495798),
 (1600,
  0.49833333333333335,
  0.4991496598639456,
  0.9783333333333334,
  0.661036036036036),
 (1700, 0.49833333333333335, 0.49914236706689535, 0.97, 0.6591166477916195),
 (1800, 0.49916666666666665, 0.4995685936151855, 0.965, 0.6583285957930642),
 (2500, 0.5, 0.5, 0.9616666666666667, 0.6579247434435576)]